In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from math import sin, cos, sqrt, atan2, radians
import geopy.distance
from sklearn import preprocessing
from scipy import interpolate
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
# First, import the data:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df_1 = pd.read_csv('fahrzeiten_soll_ist_20160103_20160109.csv')

####################
df_haltepunkt = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltepunkt.csv")

# Bus Station
df_haltestelle = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltestelle.csv")
####################

def find_dist(lat1, lon1, lat_2, lon2):
    c1 = (lat1, lon1)
    c2 = (lat_2, lon2)
    return geopy.distance.vincenty(c1, c2).km
####################
# Some useful atts #
####################

# Target attributes - from: in seconds
target_arrival_from = "soll_an_von"

# Actual attributes - from: in seconds
actual_arrival_from = "ist_an_von"

# nominal departure from in seconds
target_departure_from = "soll_ab_von"

# actual  departure from in seconds
actual_departure_from = "ist_ab_von"
#######################################

#######################################
# Target attributes - from: in seconds
target_arrival_to = "soll_an_nach"

# Actual attributes - from: in seconds
actual_arrival_to = "ist_an_nach1"

# nominal departure from in seconds
target_departure_to = "soll_ab_nach"

# actual  departure from in seconds
actual_departure_to = "ist_ab_nach"
#####################################

#####################################


In [2]:
len(set(df_1['halt_id_nach']))

659

In [3]:
df_1 = df_1[df_1['betriebsdatum'] == '05.01.16']

In [4]:
df_1['halt_punkt_id'] = df_1['halt_punkt_id_nach']
df_1 = df_1.merge(df_haltepunkt)

In [5]:
b = df_1.sort_values(by = ['betriebsdatum', 'soll_ab_von', 'fahrt_id', 'seq_von'])
b = b[b['betriebsdatum'] == b['datum_nach']]
# b

In [6]:
b = b[(b['soll_an_nach'] > 18000) & (b['soll_an_nach'] < 86400)]
# b

In [7]:
# encode labels
le = preprocessing.LabelEncoder()
le.fit(b['GPS_Latitude'])
b['Stop_id'] = le.transform(b['GPS_Latitude'])
# b

In [8]:
count = b['GPS_Latitude'].value_counts()
# count_new = count[count < 50]
# len(count_new)

In [9]:
time_period = 600
starting_time = 18000
# indicates 30 mins intervals
# Number of bins:
n_bins = int(np.floor((86400 - starting_time)/time_period))
labels = [i for i in range(n_bins - 1)]
# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins)]
b['intervals'] = pd.cut(b[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels)

In [10]:
# Some Signals
target_stationary = 'target_stationary'
target_travel_time = 'target_travel_time'
delay = 'delay'

b[target_stationary] = b[target_departure_from] - b[target_arrival_from]
b[target_travel_time] = b[target_arrival_to] - b[target_departure_from]
b[delay] = b[actual_arrival_from] - b[target_arrival_from]

b[target_stationary] = b[target_departure_from] - b[target_arrival_from]
# b['actual'] = b[actual_arrival_to] - b[actual_departure_from]

In [11]:
b.fahrweg_id.value_counts()[:5]

18305    4607
18306    4479
21424    4464
21423    4280
18348    3941
Name: fahrweg_id, dtype: int64

In [12]:
# The most amount of datapoints
temp = b.copy(deep=True)

temp = b[b['fahrweg_id'] == 18305]
# temp

In [13]:
# Discarding delays above 
temp = temp[temp['delay'] < 900]

In [14]:
%matplotlib
plt.scatter(temp[target_arrival_from], temp['delay'])
plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)))
print(len(set(temp['intervals'])))

Using matplotlib backend: Qt5Agg


NameError: name 'x' is not defined

In [15]:
# agg delays to make a signal
agg_delays = []
for i in range(len(labels)):
    agg_delays.append(temp[temp['intervals'] == i]['delay'].mean())
agg_delays = np.array(agg_delays)

In [16]:
%matplotlib
plt.plot(range(len(agg_delays)), agg_delays, '-o')
plt.xlim(auto=True)

Using matplotlib backend: Qt5Agg


(-5.6000000000000005, 117.6)

In [17]:
X = np.isnan(agg_delays)
X = np.where(X == False)
new_agg = agg_delays[~np.isnan(agg_delays)]
new_agg.shape

(113,)

In [18]:
# Interpolate

f = interp1d(X[0], new_agg, fill_value="extrapolate")

new_X = np.array(range(len(agg_delays)))

results = f(new_X)
# results

In [19]:
%matplotlib
plt.plot(new_X, results, '-o')
plt.xlim(auto=True)

Using matplotlib backend: Qt5Agg


(-5.6000000000000005, 117.6)

In [20]:
using_pd = pd.Series(agg_delays)
using_pd.interpolate(inplace=True)
using_pd.interpolate(inplace=True)
# using_pd

In [21]:
%matplotlib
plt.plot(new_X, using_pd, '-o', new_X, results, '-o')
plt.xlim(auto=True)

Using matplotlib backend: Qt5Agg


(-5.6000000000000005, 117.6)